In [63]:
from lightfm_recomender import LightfmRecomender
import os
%load_ext autoreload
%autoreload 2
new_directory = '/Users/vasevooo/projects/GameBuddy'
os.chdir(new_directory)
import pandas as pd 
import requests
import json
from sklearn.preprocessing import MinMaxScaler, StandardScaler
import numpy as np
from lightfm import LightFM
from scipy.sparse import csr_matrix, save_npz, load_npz

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### обновление предобработанных данных для модели 

In [181]:
def download_file_from_drive(file_url, save_path):
    if os.path.isfile(save_path):
        print(f"File '{save_path}' already exists.")
        return

    try:
        response = requests.get(file_url)
        response.raise_for_status()
    except requests.exceptions.RequestException as e:
        print(f"Error downloading file from '{file_url}': {str(e)}")
        return

    with open(save_path, 'wb') as file:
        file.write(response.content)
    
    print(f"File downloaded successfully: '{save_path}'")

In [182]:
file_url = 'https://drive.google.com/file/d/1tt1DxIxKtJueQd81Wa6XibM1vDuuje9o/view?usp=sharing' ## Ссылка на гугл драйв от Вани
save_path = 'data/final_gametime_df.csv'
download_file_from_drive(file_url, save_path)

File 'data/final_gametime_df.csv' already exists.


In [159]:
ru_ids = pd.read_csv('data/final_gametime_df.csv')
transformed_data = []

for index, row in ru_ids.iterrows():
    user_id = row[-1]  
    
    for game_ids, ranking in row.items():
        if game_ids != 'steamid':

            transformed_data.append([user_id, game_ids, ranking])

transformed_df = pd.DataFrame(transformed_data, columns=['user_id', 'game_id', 'ranking']).astype(int)
transformed_df.head(2)

,user_id,game_id,ranking
0,76561198256283584,10,0
1,76561198256283584,20,0


In [160]:
scaler = MinMaxScaler(feature_range=(0, 100))

transformed_df['ranking'] = transformed_df.groupby('game_id')['ranking'].transform(lambda x: scaler.fit_transform(x.values.reshape(-1, 1)).flatten())
transformed_df['ranking'] = pd.to_numeric(transformed_df['ranking'])

In [161]:
## сохранение df по юзерам для дальнейшего использования

transformed_df.to_csv('data/for_models/users_games_df.csv', index=False)

In [162]:
games = pd.read_csv('data/game_info_with_scores.csv')
item_dict ={}
games = games[['appid', 'Name']].sort_values('appid').reset_index()
games['appid'] = games['appid'].astype(int)

for i in range(games.shape[0]):
    item_dict[int(games.loc[i,'appid'])] = games.loc[i,'Name']

In [163]:
## сохранение словаря по играм для дальнейшего использования

with open("data/for_models/item_dict.json", "w") as outfile:
    json.dump(item_dict, outfile, indent=4)

### Тест класса LigthFM Recomender

In [173]:
item_dict_path = 'data/for_models/item_dict.json'
users_games_df_path = 'data/for_models/users_games_df.csv'

lfr = LightfmRecomender(users_games_df_path, item_dict_path)

In [184]:
steamid = 76561198156258305

In [185]:
new_user = lfr.get_user_games(steamid)

In [186]:
matrix, df = lfr.get_csr_matrix_for_all_users(new_user)

In [187]:
model = lfr.fit_model(matrix, epochs=100)

In [189]:
lfr.sample_recommendation_user(model, df, steamid, threshold = 0, nrec_items = 10)

User: 76561198156258305
Recommended Items:
1- Battlerite
2- Hades
3- Halo Infinite
4- Albion Online
5- Magicka
6- Soulstone Survivors
7- Northgard
8- STAR WARS™: The Old Republic™
9- DARK SOULS™ III
10- Last Epoch

 Known Likes:
1- Torchlight: Infinite
2- EA SPORTS™ FIFA 23
3- Vampire Survivors
4- V Rising
5- God of War
6- The Callisto Protocol™
7- Forza Horizon 4
8- ELDEN RING
9- Gunfire Reborn
10- Potion Craft: Alchemist Simulator
11- Gears Tactics
12- Sea of Thieves 2023 Edition
13- Apex Legends™
14- Total War: WARHAMMER III
15- Curse of the Dead Gods
16- Cyberpunk 2077
17- Destiny 2
18- Ori and the Will of the Wisps
19- Dota Underlords
20- Mortal Kombat 11
21- Among Us
22- Back 4 Blood
23- Age of Empires II: Definitive Edition
24- DOOM Eternal
25- Gloomhaven
26- Ring of Elysium
27- UnderMine
28- Slay the Spire
29- Disco Elysium - The Final Cut
30- Quake Champions
31- Realm Grinder
32- Graveyard Keeper
33- Hunt: Showdown
34- Dead Cells
35- PUBG: BATTLEGROUNDS
36- Alien Swarm: Reacti